In [2]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/data/docs/JupyterSheets.json', scope)
gc = gspread.authorize(credentials)

In [5]:
spreadsheet_key = '1KQwNxNVYJ21K1LrDcAddEj1_yxSWlxVpc6xS9z542yY'
book = gc.open_by_key(spreadsheet_key)
UKTxns_worksheet = book.worksheet("UKTxns")
table = UKTxns_worksheet.get_all_values()